### Verification of plug flow reactor RHS
* This code uses cantera to verify CThem implementation

In [1]:
import cantera as ct
import numpy as np

### Equation to solve in pfr

* species equation (eq 9.6)

$$\rho u A_c \frac{ d  Y_k }{dz} = A_c \dot{w_{k}}W_k + P^{'} \dot{s}_kW_k - Y_k P^{'}\sum_{k=1}^{Kg} \dot{s}_kW_k  $$ 

* energy equation (eq 9.76)

$$  \rho u A_c c_p\frac{dT}{dz} =  - A_c \sum_{k=1}^{Kg} \dot{w_{k}} W_k h_k - P^{'}\sum_{k=1}^{Kg}\dot{s}_kW_k h_k + \hat{h}P(T_w -T) $$ 

* momemtum equation
$$\Big(1  - \frac{ p }{\rho u^2} \Big)\rho u A_c \frac{ d  u }{dz} = - \Big( 1 + \frac{p }{\rho u^2} \Big)uP^{'} \sum_{k=1}^{Kg}\dot{s}_kW_k - \tau_w P  - A_c \rho R \big ( \frac{ 1 }{\bar{W}} \frac{dT}{dz} +    T \sum _{k=1}^{Kg} \frac{d Y_k}{ dz}\frac{1}{W_k} \big ) $$

* continuity equation 
$$ u A_c \frac{d \rho }{ dz} = - u \rho \frac{dA_c}{dz} + P^{'}\sum_{k=1}^{Kg} \dot{s}_kW_k   - \rho A_c \frac{du}{dz}$$


* surface species equation

$\dot{s}_k = 0$ k is only surface species

In [2]:
def rhs(Y,t):
    Yk      = Y[2:-1]
    u       = Y[-1]
    T       = Y[1]
    ρ       = Y[0]
    gas.Y   = np.copy(Yk)
    mix_W = gas.mean_molecular_weight
    
    #compute pressure
    P = R*T*ρ/mix_W

    gas.TP  = T, P
    surf.TP = T, P

    dAcdz = 0       
    
    ## species equations 

    #surf.advance_coverages(100)

    term1 = -Yk*Pr*np.sum(surf.net_production_rates[:Ngas]*gas.molecular_weights)
    term2 = Ac*gas.net_production_rates*gas.molecular_weights
    term3 = Pr*surf.net_production_rates[:Ngas]*gas.molecular_weights
    num1  = ρ*u*Ac
    dydz  = (term1 + term2 + term3)/num1 #np.zeros(Ngas)#
    
    ## energy equation
    hk   = gas.partial_molar_enthalpies/gas.molecular_weights # J/kg
    te1  = - np.sum(term2*hk)
    te2  = - np.sum(term3*hk)
    num2 = num1*gas.cp_mass 
    dTdz = (te1+te2)/num2 
        
    # mom equation
    m = 1. - P/(ρ*u*u) 
    tm1 = -(-m+2.)*u*Pr*np.sum(surf.net_production_rates[:Ngas]*gas.molecular_weights)
    tm2 = - Ac*ρ*R*(dTdz/mix_W + T*np.sum(dydz/gas.molecular_weights))
    numm = m*ρ*u*Ac 

        
    dudz = (tm1+tm2)/numm 
    ##continuity equation
    dρdz = (-u*ρ*dAcdz + Pr*np.sum(surf.net_production_rates[:Ngas]*gas.molecular_weights)-ρ*Ac*dudz)/(u*Ac)
    f = surf.net_production_rates[Ngas:]
    f[-1] = 1- np.sum(surf.coverages)
    
    return np.hstack((dTdz,dydz,dρdz,dudz,f))

In [3]:
mec = 'X'
# mec = 'PT'
TchemRepo = '/Users/odiazib/CODE/TChem++/'
dirInputs = TchemRepo + 'install/example/data/plug-flow-reactor/'+mec+'/'

In [4]:
R = 8.31446261815324*1000 # m3⋅Pa⋅K−1⋅Kmol−1
Ti =1300 # K
Pi = ct.one_atm

Area = 0.00053 ## // [m^2]

ui  = 0.019 # m/s
d   = (Area/np.pi)**0.5# m
Pr  = 2.*d
Ac  = Area#np.pi*d*d/4.
#0.13149028338481544
print('Pr',Pr)

Pr 0.025977239243415308


In [5]:
# import the gas model and set the initial conditions                                           
gas  = ct.Solution(dirInputs+'chem.cti', 'gas')
surf = ct.Interface(dirInputs+'chem.cti', 'surface1', [gas])
if mec == 'PT':    
    gas.TPX = Ti, Pi, 'CO:0.2, O2:0.2, N2:0.6' # X mole fraction Y mass fraction
elif mec == 'X':
    gas.TPX = Ti, Pi, 'CH4(2):0.2, O2(3):0.2, N2:0.6' # X mole fraction Y mass fraction
    #gas.Y +=1e-6
# import the surface model
surf.TP = Ti, Pi
Ngas = gas.n_species
ρi  = np.copy(gas.density)
Yki = np.copy(gas.Y)
print('ρi',ρi)
      
surf.coverages = 1./surf.n_species*np.ones(surf.n_species)
print(surf.coverages)
print(gas.Y)
# print('CO index',gas.species_index('CO'))
# print('N2 index',gas.species_index('N2'))
# print('O2 index',gas.species_index('O2'))

Nsurf = surf.n_species
Ngas  = gas.n_species
Yi = np.hstack((ρi,Ti,Yki,ui))

ρi 0.24763540348846827
[0.05882353 0.05882353 0.05882353 0.05882353 0.05882353 0.05882353
 0.05882353 0.05882353 0.05882353 0.05882353 0.05882353 0.05882353
 0.05882353 0.05882353 0.05882353 0.05882353 0.05882353]
[0.         0.         0.63627474 0.12146061 0.24226465 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]


In [6]:
state = np.vstack((gas.density,gas.P,gas.T,np.atleast_2d(gas.Y).T))
Zk  = np.copy(surf.coverages)
np.savetxt(dirInputs + 'inputGas.dat',state)
np.savetxt(dirInputs + 'inputSurfGas.dat',Zk)
np.savetxt(dirInputs + 'inputVelocity.dat',np.atleast_2d(ui))

In [7]:
rhsPRF = rhs(Yi,0)

In [8]:
nameRHS = ['T']
nameRHS += gas.species_names
nameRHS += 'dρ'
nameRHS += 'du'
nameRHS += surf.species_names

In [9]:
rhsTchem = np.loadtxt(dirInputs+'/plugflowreactorRHS.dat')

In [10]:
for i, rhs in enumerate(rhsPRF):
    print(i, nameRHS[i], rhs, rhsTchem[i][1], (rhs-rhsTchem[i][1])/(rhs+1e-23) )

0 T 9.355040778462248e+17 9.355056e+17 -1.6270947516726987e-06
1 Ar 0.0 0.0 0.0
2 Ne 0.0 0.0 0.0
3 N2 -352949953021771.3 -352950700000000.0 -2.1163856866738936e-06
4 CH4(2) -67354828115906.34 -67354970000000.0 -2.10651704748012e-06
5 O2(3) -134387384891765.3 -134387700000000.0 -2.344775404007683e-06
6 CO2(4) 129456973.36031066 129457100.0 -9.782376804740986e-07
7 H2O(5) 436897988.1393678 436900700.0 -6.207079697780848e-06
8 H2(6) 336960.62600105954 336965.4 -1.4167824285989218e-05
9 CO(7) 554687492334678.1 554688600000000.0 -1.9969177909760317e-06
10 C2H6(8) 2941478.2828281345 2941561.0 -2.812095277004241e-05
11 CH2O(9) 0.0 0.0 0.0
12 CH3(10) 17433004.94059441 17433130.0 -7.173714802324535e-06
13 C3H8(11) 0.0 0.0 0.0
14 H(12) 0.10609711932785909 0.1060986 -1.3955818501866952e-05
15 C2H5(13) 0.0 0.0 0.0
16 CH3OH(14) 156841938.24356744 156843800.0 -1.1870271774325148e-05
17 HCO(15) 0.0 0.0 0.0
18 CH3CHO(16) 0.0 0.0 0.0
19 OH(17) 3264.761171421316 3264.762 -2.5379457818545806e-07
20 C2H4(